# Introduction to Data Science - Homework 8
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

Due: Friday, March 16, 11:59pm.

In this homework, you will use clustering, regular expressions, and natural language processing. 

## Your Data
First Name:Travis
<br>
Last Name:Tiner
<br>
E-mail:u0769566@utah.edu
<br>
UID:u0769566
<br>

In [2]:
# imports and setup 

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, AgglomerativeClustering

from sklearn import tree, svm, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, KFold
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.corpus import stopwords

import re

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

## Part 1: Analyze US Crime data

We'll analyze a dataset describing 1973 violent crime rates by US State. The crimes considered are assault, murder, and rape. Also included is the percent of the population living in urban areas.

The dataset is available as *USarrests.csv*. The dataset has 50 observations (corresponding to each state) on 4 variables: 
1. Murder: Murder arrests (per 100,000 residents)
2. Assault: Assault arrests (per 100,000 residents)
3. UrbanPop: Percent urban population
4. Rape: Rape arrests (per 100,000 residents)


You can read more about the dataset [here](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/USArrests.html). 

Our goal will be to use clustering tools to understand how violent crimes differ between states. 


### Task 1.1 Import the data and perform some prelimary exploratory analysis. 
Use the *read_csv* pandas function to import the data as a dataframe. 

Plot a scatterplot matrix of the data. Explore basic statistics of the data. Write a few sentences describing how the variables are correlated. 

In [76]:
df = pd.read_csv('USarrests.csv')

In [80]:
df['State'] = df['Unnamed: 0']

In [90]:
df = df[['State','Murder','Assault','UrbanPop','Rape']]

In [92]:
df.plot.scatter(x="State",y="Assault")

ValueError: scatter requires x column to be numeric

**Your description:** TODO 

### Task 1.2 k-means cluster analysis
1. Scale the dataset using the *scale* function of the sklearn.preprocessing library. 
+ Using k-means, cluster the states into four clusters. Which states belong to which clusters?
+ Vary k and find the *best* value. How do you determine *best*? 

In [4]:
# Your code here


**Interpretation:** TODO

### Task 1.3 Hierarchical cluster analysis

1.  Using hierarchical clustering with complete linkage and Euclidean distance, cluster the states into four clusters. Which states belong to which clusters? 
2. Do you get similiar results as for k-means? 

In [5]:
# Your code here


**Interpretation:** TODO

# 2. Regular Expressions 

Write regular expressions for the following examples that matches the data of the given format and any other reasonable variations thereof. E.g., your regex shouldn't be specific to one URL or one phone number, but should work for all examples of the same format.

**2.1.** Writes a regular expression that extracts the urls out of this string, but only the URLs.

In [6]:
text = """To learn about pros/cons of data science, go to http://datascience.net.\
Alternatively, go to datascience.net/2018/"""

In [7]:
re.findall(r'[\w:/\w]+[.][\w/\-:]+', text)

['http://datascience.net', 'datascience.net/2018/']

**2.2.** Write a regular expression that extracts all phone numbers and fax numbers from this text: 

In [47]:
text = """You can reach me at 801-774-4321, or my office at (801) 223 9572.\ 
Send me a fax at 857 188 7422. We finally made the sale for all 977 giraffes.\
They wanted 225 957 dollars for it."""

In [49]:
re.findall(r'\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4}',text)

['801-774-4321', '(801) 223 9572', '857 188 7422']

**2.3.** Write a regular expression that extracts all opening html tags from this, including `<br />`.

In [11]:
html = "This is <b>important</b> and <u>very</u><i>timely</i><br />. Was this <span> what you meant?</span>"

In [12]:
re.findall(r'\<\w+\s?\/?\>',html)

['<b>', '<u>', '<i>', '<br />', '<span>']

**2.4.** Write a regular expression that extracts all the names of people from the following text. 

In [13]:
text = """Arnold Schwarzenegger was born in Austria. He and Sylvester Stalone used to run a restaurant\
with J. Edgar Hoover."""

In [20]:
re.findall(r'([A-Z]?\.?\s?[A-Z][a-z]+\s[A-Z][a-z]+)',text)

['Arnold Schwarzenegger', ' Sylvester Stalone', 'J. Edgar Hoover']

**2.5.** Write a regular expression that extracts the text out of all html elements of class important.

In [50]:
text = """Lorem ipsum dolor <b>sit</b> amet, <b class="important">consectetur adipiscing</b> elit,\ 
sed do eiusmod <span id="note">tempor incididunt ut</span> <div>labore <strong class=important>\
et dolore magna</strong> aliqua.</div> Ut enim ad minim veniam, quis nostrud exercitation ullamco."""

In [61]:
re.findall(r'class=["]?important["]?>(.*?)<\/.+>',text)

['consectetur adipiscing', 'et dolore magna']

## 3. NLP: Classifying Newsgroups Articles

Newsgroups were the social media of the 90s. Newsgroups are open discussion forums structured into hierarchices. For example, the following newsgroups cover topics as divers as atheism, computer graphics, and classified ads.  

```
alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
```

We will be combining machine learning and natural language processing to classify the news articles into these groups. We expect, for example, that the text for a classified ad in `misc.forsale` is different from text in `alt.atheism`. 

We will use the 20 Newsgroups corpus from scikit-learn. The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics. The general steps we follow are:
1. Load the corpus    
+ Do preprocessing: removal of stopwords, stemming, etc.
+ Vectorize the text
+ Split into training and test sets
+ Train our classifier

Refer to documentation on the [20 newsgroups dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) to learn about the dataset and find out how to download it.
We recommended you use the `subset='all'` parameter to load all the data at once, instead of `subset='train'` and `subset='test'` seperately.

**3.1.** Load the dataset.

1. Print the exact number of news articles in the corpus.
2. Print all 20 categories the news articles belong to.

In [ ]:
# your solution

### 3.2 Classification

Vectorize the data using vectorizers from sklearn. Using these vectors as features and the article category from corpus as labels, train a NaiveBayes classifer to classify the data.

#### Vectorizers

Vectorizes help us to transform text data into features we can use in machine learning. We did the vectorization manually in class, here you will use pre-build vectorizers. 

You should use CountVectorizer and TfidfVectorizer vectorizers from sklearn to vectorize your data. Please refer to documentation on both to learn how to use them.
+ http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
+ http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Compare the performance of classifiers using both vectorizers. You are encouraged to experiment with different parameters like max_df, min_df, etc. See docs for the meanings.

#### Naive Bayes
**Resources**
1. https://en.wikipedia.org/wiki/Naive_Bayes_classifier
2. https://www.geeksforgeeks.org/naive-bayes-classifiers
3. http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

We will be using Multinomial Naive Bayes from sklearn. Refer to documentation above for how to import the classifer. Then it can be used like any other classifer by using fit and predict functions provided on it.
e.g:

```
clf = MulitnomialNB(alpha = 1)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
```

Alpha is also known as the smoothing factor and ranges from 0 (no smoothing) to 1 (Laplace Smoothing). You can experiment with different values to see if you get better results. 


In [ ]:
# your solution

#### Removing Stopwords

Now we'll use the NLTK stopword list to improve our data vectors. TfidfVectorizer and CountVectorizer both can take an argument called stop_words. The words passed to this arguement are considered as stopwords and are not vectorized. Use the stopwords list from NLTK and pass it to vectorizers. Then evalulate the new vectors using Multinomial Naive Bayes.

In [ ]:
# your solution

#### Interpretation

1. How much accuracy would a naive approach, that picks one of the 20 categories at random achieve?
1. What accuracy where you able to achieve? 
1. What was the influence of the different vectorizers and the stopword removal? 


**Your Reponses:** TODO